In [72]:
import os
os.environ['NVIDIA_VISIBLE_DEVICES'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [37]:
import random
import copy
from tqdm import tqdm
import torch
import json
from dotenv import load_dotenv

load_dotenv()

### OpenAI Access

In [11]:
import os
from openai import AzureOpenAI


class OpenAILLM:
    def __init__(self, model):
        self.__openai_client = AzureOpenAI(
            api_key=os.getenv("API_KEY"),
            api_version=os.getenv("API_VERSION"),
            azure_endpoint=os.getenv("AZURE_ENDPOINT")
        )
        self.__model = model

    def get_completion(self, prompt, temperature=0, max_tokens=500):
        messages = [{"role": "user", "content": prompt}]
        response = self.__openai_client.chat.completions.create(
            model=self.__model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return response.choices[0].message.content

    def __call__(self, prompt, temperature=0, max_tokens=500):
        return self.get_completion(prompt, temperature, max_tokens)

In [12]:
openai_llm = OpenAILLM(model="gpt-4")

### Generate ambiguous utterances

In [3]:
utterances = [
    "Set the mood for a cozy dinner setting.",
    "Play some relaxing music for a spa-like atmosphere.",
    "It's getting too hot in here.",
    "Start the dishwasher after dinner is finished.",
    "Set the mood for a romantic evening at home.",
    "Tidy up the house.",
    "Adjust the temperature for a comfortable reading session.",
    "Start the laundry day.",
    "Set the ambience for a productive work session.",
    "Turn on the karaoke night.",
    "Adjust the temperature for a good night's sleep.",
    "Set the mood for a family game night.",
    "Set the ambience for a cozy afternoon nap.",
    "Adjust the temperature for a cozy winter evening.",
    "Play some energetic music for a cleaning session.",
    "Adjust the lights for a bright and productive day.",
    "Set the mood for a romantic dinner at home.",
    "Turn on the AC for a comfortable afternoon.",
    "Set the ambience for a quiet evening of reading.",
]

In [7]:
st_devices = {
    "airconditioner" : ["device_turnOff","device_turnOn","airConditioningMode_getValue","airConditioningMode_setValue","deviceTemperatureSetting_setValue",
    "deviceTemperatureSetting_getValue","deviceTemperatureSetting_increase","deviceTemperatureSetting_decrease"],

    "airdresser" : ["dressing_cancel","dressing_getProgress","dressing_getRemainingTime","dressing_getStatus","dressing_getSupportedCourses","dressingMode_setValue","dressing_pause","dressing_start","dressing_startCourse"],

    "blind" : ["device_open", "device_openPartial", "device_close", "device_closePartial"],

    "shoedresser" : ["dressing_cancel","dressing_getProgress","dressing_getRemainingTime","dressing_getStatus","dressing_getSupportedCourses","dressingMode_setValue","dressing_pause","dressing_start","dressing_startCourse"],

    "airpurifier" : ["device_turnOff","device_turnOn","mode_setValue","mode_getValue"],

    "boiler" : ["device_turnOff","device_turnOn","deviceTemperatureSetting_setValue","deviceTemperatureSetting_getValue","deviceTemperatureSetting_increase",
    "deviceTemperatureSetting_decrease"],

    "camera" : ["device_turnOff","device_turnOn","camera_show"],

    "cooktop" : ["device_turnOff","device_turnOn","cooking_start","cooking_startCourse","deviceTemperatureSetting_setValue","deviceTemperatureSetting_getValue",
    "deviceTemperatureSetting_increase","deviceTemperatureSetting_decrease"],

    "dehumidifier" : ["device_turnOff","device_turnOn","deviceTemperatureSetting_setValue","deviceTemperatureSetting_getValue",
    "deviceTemperatureSetting_increase","deviceTemperatureSetting_decrease"],

    "dishwasher" : ["dishWashing_cancel","dishWashing_getProgress","dishWashing_getRemainingTime","dishWashing_getStatus","dishWashing_getSupportedCourses","dishWashing_pause","dishWashing_setCourse","dishWashing_start","dishWashing_startCourse"],

    "dryer" : ["drying_cancel","drying_getProgress","drying_getRemainingTime","drying_getStatus","drying_getSupportedCourses","dryingLevel_decrease","dryingLevel_increase","dryingLevel_setValue","drying_pause","drying_setCourse","drying_start","drying_startCourse","dryingTime_decrease","dryingTime_increase","dryingTime_setValue"],

    "fan" : ["device_turnOff","device_turnOn"],

    "humidifier" : ["device_turnOff","device_turnOn","deviceTemperatureSetting_setValue","deviceTemperatureSetting_getValue","deviceTemperatureSetting_increase",
    "deviceTemperatureSetting_decrease"],

    "light" : ["device_turnOff","device_turnOn","brightness_decrease","brightness_getValue","brightness_increase","brightness_setValue","colorControl_setValue", "brightness_decreasePartial", "brightness_increasePartial", "brightness_setMax", "brightness_setMin"],

    "microwave" : ["cooking_getProgress","cooking_start","cooking_startCourse","cookingTemperature_decrease","cookingTemperature_getValue","cookingTemperature_increase","cookingTemperature_setValue","microwave_start","mode_getValue"],

    "oven" : ["cooking_getProgress","cooking_start","cooking_startCourse","cookingTemperature_decrease","cookingTemperature_getValue","cookingTemperature_increase","cookingTemperature_setValue","mode_getValue"],

    "projector" : ["brightness_decrease","brightness_getValue","brightness_increase","brightness_setValue","colorControl_setValue",
    "device_turnOff","device_turnOn","media_play","media_pause","media_stop","mute_turnOn","mute_turnOff","volume_increase","volume_getValue",
    "volume_decrease", "volume_setHalf", "volume_setMax", "volume_setMin", "brightness_decreasePartial", "brightness_increasePartial",
    "brightness_setMax", "brightness_setMin"],

    "refrigerator" : ["device_turnOff","device_turnOn","deviceTemperatureSetting_setValue","deviceTemperatureSetting_getValue",
    "deviceTemperatureSetting_increase","deviceTemperatureSetting_decrease"],

    "ricecooker" : ["cooking_getProgress","cooking_start","cooking_startCourse","cookingTemperature_decrease","cookingTemperature_getValue","cookingTemperature_increase","cookingTemperature_setValue"],

    "robotcleaner" : ["cleaning_getProgress","cleaning_getStatus","cleaningMap_create","cleaningMap_remove","cleaningOption_turnOff","cleaningOption_turnOn","cleaning_pause","cleaning_setMode","cleaning_start","cleaning_startExclude","cleaning_stop"],
    
    "speaker" : ["device_turnOff","device_turnOn","volume_increase","volume_getValue","volume_decrease", "volume_setHalf", "volume_setMax", "volume_setMin"],

    "steamcloset" : ["dressing_cancel","dressing_getProgress","dressing_getRemainingTime","dressing_getStatus","dressing_getSupportedCourses","dressingMode_setValue","dressing_pause","dressing_start","dressing_startCourse"],

    "stickcleaner" : ["cleaning_getProgress","cleaning_getStatus","cleaningMap_create","cleaningMap_remove","cleaningOption_turnOff","cleaningOption_turnOn","cleaning_pause","cleaning_setMode","cleaning_start","cleaning_startExclude","cleaning_stop"],

    "thermostat" : ["device_turnOff","device_turnOn","deviceTemperatureSetting_setValue","deviceTemperatureSetting_getValue","deviceTemperatureSetting_increase",
    "deviceTemperatureSetting_decrease"],

    "tv" : ["channel_getValue","channel_setByName","application_exit","application_launch","application_searchIn","brightness_decrease",
    "brightness_getValue","brightness_increase","brightness_setValue","channel_setByNumber","channel_setPrevious","colorControl_setValue",
    "device_turnOff","device_turnOn","media_play","media_pause","media_stop","mute_turnOn","mute_turnOff","volume_increase","volume_getValue",
    "volume_decrease", "volume_setHalf", "volume_setMax", "volume_setMin", "brightness_decreasePartial", "brightness_increasePartial", "brightness_setMax", "brightness_setMin", "channel_setByProgram"],
    
    "washer" : ["washing_pause","washing_setCourse","washing_start","washing_startCourse","washingTemperature_decrease","washingTemperature_increase","washingTemperatureMode_setValue","washingTemperature_setColdWater","washingTemperature_setHotWater","washingTemperature_setValue"],

    "waterpurifier" : ["device_turnOff","device_turnOn","deviceTemperatureSetting_setValue","deviceTemperatureSetting_getValue",
    "deviceTemperatureSetting_increase","deviceTemperatureSetting_decrease"]

}

In [6]:
prompt_template = """You are given examples of some ambiguous commands that a user can give to a home assistant. You need to generate 200 more such ambiguous \
commands that a user can give to a home assistant. Here are the devices a home assistant has access to:

Available Devices: {devices}

Example ambiguous commands:
{commands}

Now generate 200 such commands. The commands have to be valid and executable using any combination of the available devices. Your output format should \
be a python list."""

In [13]:
prompt = prompt_template.format(devices=sorted(st_devices), commands="\n".join([f"{i+1}) {utterances[i]}" for i in range(len(utterances))]))
print(prompt)

You are given examples of some ambiguous commands that a user can give to a home assistant. You need to generate 200 more such ambiguous commands that a user can give to a home assistant. Here are the devices a home assistant has access to:

Available Devices: ['airconditioner', 'airdresser', 'airpurifier', 'blind', 'boiler', 'camera', 'cooktop', 'dehumidifier', 'dishwasher', 'dryer', 'fan', 'humidifier', 'light', 'microwave', 'oven', 'projector', 'refrigerator', 'ricecooker', 'robotcleaner', 'shoedresser', 'speaker', 'steamcloset', 'stickcleaner', 'thermostat', 'tv', 'washer', 'waterpurifier']

Example ambiguous commands:
1) Set the mood for a cozy dinner setting.
2) Play some relaxing music for a spa-like atmosphere.
3) It's getting too hot in here.
4) Start the dishwasher after dinner is finished.
5) Set the mood for a romantic evening at home.
6) Tidy up the house.
7) Adjust the temperature for a comfortable reading session.
8) Start the laundry day.
9) Set the ambience for a produ

In [23]:
response = openai_llm.get_completion(prompt, max_tokens=4000)

In [24]:
print(response)

Given the complexity and the requirement for a high number of entries, I'll provide a substantial list that follows the spirit of the examples you've provided. This list will cover a wide range of activities and scenarios, keeping in mind the devices available for a home assistant to control. However, reaching exactly 200 unique, contextually rich, and ambiguous commands while ensuring no repetition and maintaining quality is a challenging task. Here's a list that should serve a similar purpose:

```python
ambiguous_commands = [
    "Prepare the house for a movie marathon.",
    "Make it feel like summer in the middle of winter.",
    "Create a perfect environment for studying.",
    "Get the house ready for a surprise party.",
    "Set up a serene yoga space.",
    "Prepare for an all-nighter with friends.",
    "Cool down the kitchen after cooking.",
    "Warm up the bathroom for a shower.",
    "Create a perfect gaming atmosphere.",
    "Set the scene for a virtual beach getaway.",


In [25]:
ambiguous_commands = [
    "Prepare the house for a movie marathon.",
    "Make it feel like spring inside.",
    "Get the place ready for a surprise party.",
    "Create a zen space for meditation.",
    "Set up for an all-night study session.",
    "Prepare a romantic setting for an anniversary.",
    "Cool down the kitchen while I cook.",
    "Warm up the bathroom for a shower.",
    "Make the living room guest-ready.",
    "Set the scene for a horror movie night.",
    "Create a bright workspace.",
    "Dim the lights for a relaxing bath.",
    "Freshen up the room before guests arrive.",
    "Keep the bedroom cool for sleeping.",
    "Warm up the house for a cold day.",
    "Prepare for an indoor picnic.",
    "Set up a calming nightlight for the kids.",
    "Make the house smell fresh and clean.",
    "Get the party started with some dance music.",
    "Quiet down the house for a baby's nap time.",
    "Ensure the floors are clean for a toddler's playtime.",
    "Set a reminder to water the plants.",
    "Prepare the kitchen for a baking day.",
    "Cool down the workout area.",
    "Set up a gaming tournament atmosphere.",
    "Create a cozy reading nook.",
    "Prepare the dining room for a family dinner.",
    "Set the perfect temperature for yoga.",
    "Make the house sparkle for the holidays.",
    "Get ready for a virtual meeting.",
    "Set the mood for a wine tasting evening.",
    "Prepare for a pet's comfortable stay.",
    "Set up for a home spa day.",
    "Create a festive atmosphere for a birthday.",
    "Cool the air for a summer night's sleep.",
    "Warm up for a winter morning.",
    "Set the scene for a DIY craft day.",
    "Prepare for an at-home date night.",
    "Make the environment allergy-friendly.",
    "Set up for a peaceful morning routine.",
    "Create a vibrant atmosphere for a home workout.",
    "Prepare the house for a sleepover.",
    "Set the mood for a virtual concert viewing.",
    "Make the living area cozy for a family movie night.",
    "Set up a serene space for afternoon tea.",
    "Prepare for a night of stargazing.",
    "Create a motivational environment for home office work.",
    "Set the scene for a baking marathon.",
    "Prepare a welcoming atmosphere for a book club meeting.",
    "Make the bedroom ready for a lazy Sunday.",
    "Set up a cool environment for electronic devices.",
    "Create a warm setting for a winter breakfast.",
    "Prepare for a relaxing evening with classical music.",
    "Set the mood for a creative writing session.",
    "Make the air clean and fresh for a newborn.",
    "Set up for an energizing morning wake-up.",
    "Prepare the house for a festive celebration.",
    "Create a calm atmosphere for studying.",
    "Set the scene for a romantic movie watching.",
    "Prepare for a fun-filled family game evening.",
    "Make the dining area perfect for a gourmet dinner.",
    "Set up for a quiet and reflective meditation session.",
    "Create a lively atmosphere for a home dance party.",
    "Prepare the living room for a casual hangout.",
    "Set the mood for an intimate tea party.",
    "Make the guest room welcoming for overnight guests.",
    "Set up for a productive home cleaning day.",
    "Prepare for a cozy winter evening with hot cocoa.",
    "Create a refreshing summer environment with cool air.",
    "Set the scene for an indoor gardening day.",
    "Prepare for a peaceful and restful night.",
    "Make the kitchen ideal for cooking a big meal.",
    "Set up for a sunny morning breakfast.",
    "Create a romantic ambiance for Valentine's Day.",
    "Prepare the house for a New Year's Eve party.",
    "Set the mood for an autumn harvest dinner.",
    "Make the bathroom spa-like for a relaxing soak.",
    "Set up for a high-energy fitness workout.",
    "Prepare for a quiet and cozy knitting session.",
    "Create a focused environment for tax preparation.",
    "Set the scene for a kids' indoor camping night.",
    "Prepare for a soothing and calm yoga practice.",
    "Make the home office conducive to productivity.",
    "Set up for a vibrant and fun painting party.",
    "Prepare the patio for a summer evening barbecue.",
    "Create a cool and comfortable sleep environment.",
    "Set the mood for an elegant home cocktail party.",
    "Make the nursery warm and cozy for a baby.",
    "Set up for a refreshing morning routine.",
    "Prepare for a night of binge-watching favorite shows.",
    "Create a cozy and warm atmosphere for storytelling.",
    "Set the scene for a lively and fun karaoke night.",
    "Prepare for a tranquil and serene afternoon nap.",
    "Make the game room ready for a night of video games.",
    "Set up for a festive and joyful holiday gathering.",
    "Prepare the art studio for a day of creativity.",
    "Create a cool and focused environment for homework.",
    "Set the mood for a nostalgic movie marathon.",
    "Make the sunroom bright and cheerful for breakfast.",
    "Set up for a quiet and peaceful journaling session.",
    "Prepare for a lively and energetic morning workout.",
    "Create a warm and inviting atmosphere for a family reunion.",
    "Set the scene for a sophisticated wine and cheese night.",
    "Prepare for a fun and exciting virtual reality game night.",
    "Make the deck or patio ready for a summer party.",
    "Set up for a calm and relaxing massage session.",
    "Prepare for a night of deep and restorative sleep.",
    "Create a bright and energizing atmosphere for a home office.",
    "Set the mood for a cozy and intimate fireside chat.",
    "Make the entryway welcoming for guests.",
    "Set up for a productive and focused study marathon.",
    "Prepare for a warm and cozy hot chocolate evening.",
    "Create a festive and bright atmosphere for holiday decorating.",
    "Set the scene for an elegant and romantic proposal.",
    "Prepare for a night of storytelling and marshmallow roasting.",
    "Make the backyard ready for a family barbecue.",
    "Set up for a peaceful and restful meditation session.",
    "Prepare for a lively and fun-filled children's party.",
    "Create a cool and comfortable environment for a heatwave.",
    "Set the mood for a spooky and thrilling Halloween night.",
    "Make the dining room elegant for a formal dinner.",
    "Set up for a bright and cheerful morning wake-up.",
    "Prepare for a cozy and snug movie night.",
    "Create a warm and inviting space for holiday guests.",
    "Set the scene for a quiet and reflective writing retreat.",
    "Prepare for a vibrant and energetic dance practice session.",
    "Make the living room ready for a family gathering.",
    "Set up for a cool and refreshing summer day.",
    "Prepare for a warm and comforting evening with friends.",
    "Create a festive and joyful atmosphere for a birthday celebration.",
    "Set the mood for a relaxing and calm spa day at home.",
    "Make the kitchen ready for a cooking class.",
    "Set up for a productive and organized home office day.",
    "Prepare for a night of relaxation and pampering.",
    "Create a cool and airy environment for a summer sleepover.",
    "Set the scene for a warm and cozy winter gathering.",
    "Prepare for a fun and exciting game night with friends.",
    "Make the exercise room ready for a vigorous workout.",
    "Set up for a calm and soothing yoga session.",
    "Prepare for a bright and sunny breakfast nook.",
    "Create a romantic and intimate dinner setting.",
    "Set the mood for an adventurous and fun family night.",
    "Make the guest bedroom welcoming and comfortable.",
    "Set up for a refreshing and energizing morning.",
    "Prepare for a cozy and warm evening of reading.",
    "Create a lively and fun atmosphere for a children's playdate.",
    "Set the scene for a quiet and peaceful night's sleep.",
    "Prepare for a festive and colorful holiday party.",
    "Make the bathroom ready for a luxurious bath.",
    "Set up for a cool and comfortable night during a heatwave.",
    "Prepare for a warm and cozy fireside gathering.",
    "Create a bright and productive workspace.",
    "Set the mood for a relaxing and peaceful meditation session.",
    "Make the kitchen ideal for a family baking day.",
    "Set up for a vibrant and cheerful party atmosphere.",
    "Prepare for a quiet and restful evening at home.",
    "Create a warm and inviting space for a tea party.",
    "Set the scene for a fun and energetic workout session.",
    "Prepare for a cozy and intimate movie marathon.",
    "Make the outdoor area ready for a summer evening dinner.",
    "Set up for a refreshing and cool afternoon nap.",
    "Prepare for a lively and joyous celebration.",
    "Create a calm and focused environment for studying.",
    "Set the mood for a romantic and memorable anniversary dinner.",
    "Make the living area cozy for a night in.",
    "Set up for a productive and efficient cleaning day.",
    "Prepare for a warm and festive holiday meal.",
    
    "Prepare the house for a movie marathon.",
    "Make it feel like summer in the middle of winter.",
    "Create a perfect environment for studying.",
    "Get the house ready for a surprise party.",
    "Set up a serene yoga space.",
    "Prepare for an all-nighter with friends.",
    "Cool down the kitchen after cooking.",
    "Warm up the bathroom for a shower.",
    "Create a perfect gaming atmosphere.",
    "Set the scene for a virtual beach getaway.",
    "Make the living room guest-ready.",
    "Prepare the bedroom for a lazy Sunday.",
    "Set up for an indoor date night.",
    "Get the nursery ready for bedtime.",
    "Create a calming environment for meditation.",
    "Prepare for a festive holiday dinner.",
    "Set the stage for a home concert.",
    "Make the home office more inviting for work.",
    "Cool down the workout space.",
    "Warm up the space for a winter breakfast.",
    "Create a bright environment for painting.",
    "Set a relaxing scene for bath time.",
    "Prepare the dining room for a formal dinner.",
    "Make the kids' room ready for playtime.",
    "Set up a cozy reading nook.",
    "Prepare the house for a sleepover.",
    "Create a lively atmosphere for a birthday party.",
    "Set the mood for a wine tasting evening.",
    "Make the garden ready for a barbecue.",
    "Prepare the living room for a family gathering.",
    "Set up a spooky atmosphere for a horror movie night.",
    "Create a romantic setting for an anniversary.",
    "Make the kitchen ideal for baking.",
    "Prepare the garage for a DIY project.",
    "Set up a comfortable environment for a newborn.",
    "Create a festive mood for a New Year's party.",
    "Make the patio ready for a summer dinner.",
    "Prepare the house for a quiet weekend.",
    "Set up for a productive morning routine.",
    "Create a cool environment for a hot day.",
    "Make the house warm and welcoming for guests.",
    "Prepare for an elegant afternoon tea.",
    "Set up a fun karaoke evening.",
    "Create a motivational atmosphere for working out.",
    "Make the bathroom feel like a spa.",
    "Prepare the kitchen for a cooking show.",
    "Set the mood for a mystery novel reading.",
    "Create a vibrant setting for a dance party.",
    "Make the bedroom ready for a peaceful night.",
    "Prepare the study for focused work.",
    "Set up a nostalgic evening with old movies.",
    "Create a quiet space for writing.",
    "Make the living room ready for a board game night.",
    "Prepare the house for a pet's comfort.",
    "Set up a warm welcome for a rainy day.",
    "Create a cool retreat for a summer day.",
    "Make the dining area ready for a brunch.",
    "Prepare the entertainment room for a sports night.",
    "Set up a cozy corner for knitting.",
    "Create a bright space for a home office.",
    "Make the guest room welcoming for visitors.",
    "Prepare the house for a weekend cleaning spree.",
    "Set up a romantic backdrop for Valentine's Day.",
    "Create a festive atmosphere for Christmas.",
    "Make the porch welcoming for an evening sit-out.",
    "Prepare the home for an energy-saving mode.",
    "Set up a lively atmosphere for a children's party.",
    "Create a serene environment for afternoon naps.",
    "Make the house ready for a fitness challenge.",
    "Prepare the living room for a family photo shoot.",
    "Set up a cozy ambiance for a winter evening.",
    "Create a refreshing atmosphere for a hot afternoon.",
    "Make the kitchen ready for a meal prep session.",
    "Prepare the bedroom for a restful sleep.",
    "Set up a motivational environment for early mornings.",
    "Create a peaceful setting for evening relaxation.",
    "Make the bathroom ready for a quick refresh.",
    "Prepare the study for a night of research.",
    "Set up a festive mood for Halloween.",
    "Create a warm atmosphere for Thanksgiving dinner.",
    "Make the backyard ready for a fire pit gathering.",
    "Prepare the house for a digital detox day.",
    "Set up a bright and cheerful morning.",
    "Create a cozy setting for a winter's day reading.",
    "Make the game room ready for a tournament.",
    "Prepare the art studio for a creative session.",
    "Set up a quiet corner for meditation.",
    "Create a lively kitchen for a cooking party.",
    "Make the living room ready for a DIY craft day.",
    "Prepare the bedroom for a beauty sleep.",
    "Set up a serene environment for a tea ceremony.",
    "Create a cool and comfortable space for a heatwave.",
    "Make the dining room ready for a candlelit dinner.",
    "Prepare the home theater for a blockbuster night.",
    "Set up a vibrant atmosphere for a virtual party.",
    "Create a calming ambiance for stress relief.",
    "Make the nursery ready for play and learning.",
    "Prepare the house for an eco-friendly mode.",
    "Set up a warm and cozy morning.",
    "Create a bright and airy space for a spring day.",
    "Make the workout room ready for a fitness session.",
    "Prepare the living room for a casual hangout.",
    "Set up a romantic scene for a proposal.",
    "Create a festive environment for Easter.",
    "Make the patio ready for a morning coffee.",
    "Prepare the kitchen for a healthy cooking session.",
    "Set up a cozy spot for winter storytelling.",
    "Create a refreshing setting for a summer evening.",
    "Make the bedroom ready for a dreamy night.",
    "Prepare the office for an efficient workday.",
    "Set up a fun atmosphere for a pajama party.",
    "Create a peaceful ambiance for a quiet day.",
    "Make the bathroom ready for a luxury bath.",
    "Prepare the game room for a family tournament.",
    "Set up a cozy fireplace setting for a chilly evening.",
    "Create a bright workspace for creative projects.",
    "Make the guest bedroom ready for the holidays.",
    "Prepare the house for a pet-friendly environment.",
    "Set up a warm kitchen for a baking day.",
    "Create a cool living room for a relaxation day.",
    "Make the dining area ready for a gourmet meal.",
    "Prepare the bedroom for an early rise.",
    "Set up a motivational space for goal setting.",
    "Create a serene garden for a morning yoga.",
    "Make the entertainment area ready for a movie premiere.",
    "Prepare the study for a productive afternoon.",
    "Set up a cozy nook for a winter tea.",
    "Create a lively atmosphere for a family reunion.",
    "Make the kitchen ready for a Sunday roast.",
    "Prepare the living room for a casual movie night.",
    "Set up a romantic dinner setting for two.",
    "Create a festive mood for a holiday celebration.",
    "Make the outdoor area ready for a summer party.",
    "Prepare the house for a quiet and restful weekend.",
    "Set up a bright and energetic morning.",
    "Create a cozy and warm evening for relaxation.",
    "Make the bathroom ready for a spa day at home.",
    "Prepare the kitchen for a healthy meal prep.",
    "Set up a fun and energetic workout session.",
    "Create a peaceful and calm sleeping environment.",
    "Make the living room ready for a family game tournament.",
    "Prepare the dining room for a special occasion dinner.",
    "Set up a cozy and inviting reading corner.",
    "Create a lively and fun dance floor at home.",
    "Make the bedroom ready for a restorative sleep.",
    "Prepare the home office for a day of focused work.",
    "Set up a refreshing and cool environment for a hot day.",
    "Create a warm and cozy atmosphere for a cold evening.",
    "Make the kitchen ready for a cooking challenge.",
    "Prepare the living room for a relaxing day at home.",
    "Set up a bright and productive workspace.",
    "Create a serene and peaceful meditation space.",
    "Make the bathroom ready for a quick morning routine.",
    "Prepare the bedroom for a cozy and lazy day.",
    "Set up a fun and exciting play area for kids.",
    "Create a warm and welcoming space for guests.",
    "Make the dining area ready for a family meal.",
    "Prepare the house for an energy-efficient night.",
    "Set up a cozy and romantic evening for two.",
    "Create a bright and cheerful atmosphere for a party.",
    "Make the outdoor space ready for a garden party.",
    "Prepare the kitchen for a baking marathon.",
    "Set up a comfortable and cool sleeping environment.",
    "Create a warm and inviting space for a winter gathering.",
    "Make the living room ready for a casual hangout with friends.",
    "Prepare the bedroom for a peaceful and restful night.",
    "Set up a productive and motivating work environment.",
    "Create a cool and refreshing space for a summer day.",
    "Make the bathroom ready for a relaxing bath time.",
    "Prepare the dining room for a romantic dinner.",
    "Set up a cozy and warm living room for a movie night.",
    "Create a bright and airy kitchen for breakfast.",
    "Make the bedroom ready for a good night's sleep.",
    "Prepare the home office for a day of meetings.",
    "Set up a fun and lively atmosphere for a children's birthday party.",
    "Create a serene and calm environment for relaxation.",
    "Make the outdoor area ready for a barbecue night.",
    "Prepare the living room for a family gathering.",
    "Set up a cozy and inviting space for a winter evening.",
    "Create a cool and comfortable bedroom for a summer night.",
    "Make the kitchen ready for a family cooking session.",
    "Prepare the bathroom for a quick and refreshing shower.",
    "Set up a productive and focused study space.",
    "Create a warm and cozy atmosphere for a chilly morning.",
    "Make the dining area ready for a special breakfast.",
    "Prepare the house for a pet's playtime.",
    "Set up a romantic and intimate setting for an anniversary.",
    "Create a festive and joyful atmosphere for a holiday party.",
    "Make the outdoor space ready for a relaxing afternoon.",
    "Prepare the kitchen for a healthy cooking day.",
    "Set up a comfortable and restful sleeping environment.",
    "Create a warm and inviting living room for a cozy night in.",
    "Make the bathroom ready for a luxury spa experience at home.",
    "Prepare the bedroom for a restful and peaceful sleep.",
    "Set up a bright and productive home office for work.",
    "Create a cool and refreshing living room for a hot day.",
    "Make the dining room ready for a family dinner.",
    "Prepare the house for a cleaning and tidying day.",
    "Set up a cozy and warm space for a winter's evening.",
    "Create a bright and cheerful atmosphere for a morning routine.",
    "Make the kitchen ready for a weekend brunch.",
    "Prepare the bedroom for a lazy and relaxing day.",
    "Set up a fun and energetic playroom for kids.",
    "Create a serene and peaceful environment for meditation.",
    "Make the living room ready for a casual movie night.",
    "Prepare the outdoor area for a summer evening gathering.",
    "Set up a cozy and romantic setting for a date night at home.",
    "Create a festive and lively atmosphere for a party.",
    "Make the bathroom ready for a refreshing morning shower.",
    "Prepare the kitchen for a gourmet cooking session.",
    "Set up a comfortable and cool bedroom for a summer night.",
    "Create a warm and cozy living room for a winter day.",
    "Make the dining area ready for a special occasion meal.",
    "Prepare the house for a pet-friendly environment.",
    "Set up a bright and productive workspace for a day of work.",
    "Create a cool and refreshing space for a hot afternoon.",
    "Make the bedroom ready for a restful and peaceful night.",
    "Prepare the living room for a family game night.",
    "Set up a cozy and inviting reading nook for a quiet evening.",
    "Create a lively and fun atmosphere for a dance party at home.",
    "Make the kitchen ready for a baking day with friends.",
    "Prepare the bathroom for a luxury spa-like experience.",
    "Set up a comfortable and restful environment for a good night's sleep.",
    "Create a warm and cozy space for a chilly morning.",
    "Make the dining room ready for a cozy breakfast.",
    "Prepare the house for a day of relaxation and leisure.",
    "Set up a bright and cheerful living room for a day at home.",
    "Create a cool and comfortable bedroom for a restful sleep.",
    "Make the kitchen ready for a healthy meal prep session.",
    "Prepare the outdoor area for a relaxing evening under the stars.",
    "Set up a cozy and romantic atmosphere for a special dinner at home.",
    "Create a festive and joyful environment for a holiday celebration.",
    "Make the bathroom ready for a refreshing and quick morning routine.",
    "Prepare the bedroom for a restful and peaceful night's sleep.",
    "Set up a productive and focused workspace for a day of work.",
    "Create a cool and refreshing living room for a hot summer day.",
    "Make the dining area ready for a family meal.",
    "Prepare the house for a spring cleaning day.",
    "Set up a cozy and warm space for a winter evening.",
    "Create a bright and cheerful atmosphere for a morning workout.",
    "Make the kitchen ready for a weekend cooking project.",
    "Prepare the bedroom for a lazy and relaxing weekend.",
    "Set up a fun and playful environment for a children's party.",
    "Create a serene and calm space for meditation and relaxation.",
    "Make the living room ready for a casual and cozy movie night.",
    "Prepare the outdoor area for a summer barbecue with friends.",
    "Set up a romantic and intimate setting for a special anniversary dinner.",
    "Create a festive and lively atmosphere for a birthday celebration.",
    "Make the bathroom ready for a spa-like experience at home.",
    "Prepare the kitchen for a gourmet cooking challenge.",
    "Set up a comfortable and cool bedroom for a restful summer night.",
    "Create a warm and cozy living room for a winter day's relaxation.",
    "Make the dining room ready for a special family dinner.",
    "Prepare the house for a pet-friendly and comfortable environment.",
    "Set up a bright and productive workspace for a day of focused work.",
    "Create a cool and refreshing space for relaxation on a hot day.",
    "Make the bedroom ready for a peaceful and restful night.",
    "Prepare the living room for a fun and engaging family game night.",
    "Set up a cozy and inviting reading corner for a quiet evening of relaxation.",
    "Create a lively and energetic atmosphere for a home dance party.",
    "Make the kitchen ready for a fun and social baking day with friends.",
    "Prepare the bathroom for a luxurious and relaxing spa experience at home.",
    "Set up a comfortable and restful sleeping environment for a good night's sleep.",
    "Create a warm and cozy atmosphere for a chilly morning.",
    "Make the dining area ready for a cozy and intimate breakfast.",
    "Prepare the house for a day of leisure and relaxation.",
    "Set up a bright and cheerful living room for a day spent at home.",
    "Create a cool and comfortable bedroom for a restful and peaceful sleep.",
    "Make the kitchen ready for a healthy and productive meal prep session.",
    "Prepare the outdoor area for a relaxing and enjoyable evening under the stars.",
    "Set up a cozy and romantic atmosphere for a special dinner at home.",
    "Create a festive and joyful environment for celebrating a holiday with loved ones.",
    "Make the bathroom ready for a quick and refreshing morning routine.",
    "Prepare the bedroom for a restful and peaceful night's sleep.",
    "Set up a productive and focused workspace for a day of efficient work.",
    "Create a cool and refreshing living room for relaxation during a hot summer day.",
    "Make the dining area ready for a family meal and gathering.",
    "Prepare the house for a thorough spring cleaning and organization day.",
    "Set up a cozy and warm space for enjoying a winter evening at home.",
    "Create a bright and cheerful atmosphere for starting the day with a morning workout.",
    "Make the kitchen ready for a weekend cooking and baking project.",
    "Prepare the bedroom for a relaxing and lazy weekend.",
    "Set up a fun and playful environment for hosting a children's birthday party.",
    "Create a serene and calm space for meditation and quiet relaxation.",
    "Make the living room ready for a casual and cozy movie night with family.",
    "Prepare the outdoor area for a summer barbecue and gathering with friends.",
    "Set up a romantic and intimate setting for celebrating a special anniversary.",
    "Create a festive and lively atmosphere for a birthday party at home.",
    "Make the bathroom ready for a luxurious and relaxing spa experience.",
    "Prepare the kitchen for a gourmet cooking challenge with friends.",
    "Set up a comfortable and cool bedroom for a restful night during the summer.",
    "Create a warm and cozy living room atmosphere for a relaxing winter day.",
    "Make the dining room ready for hosting a special family dinner.",
    "Prepare the house for a pet-friendly and comfortable environment for all.",
    "Set up a bright and productive workspace for a day of focused and efficient work.",
    "Create a cool and refreshing space for relaxation and comfort on a hot day.",
    "Make the bedroom ready for a peaceful and restful night's sleep.",
    "Prepare the living room for a fun and engaging family game night.",
    "Set up a cozy and inviting reading corner for a quiet and relaxing evening.",
    "Create a lively and energetic atmosphere for a home dance party with friends.",
    "Make the kitchen ready for a social and fun baking day.",
    "Prepare the bathroom for a luxurious and relaxing spa-like experience at home.",
    "Set up a comfortable and restful sleeping environment for a good night's rest.",
    "Create a warm and cozy atmosphere for a chilly morning at home.",
    "Make the dining area ready for a cozy and intimate breakfast with loved ones.",
    "Prepare the house for a day of leisure, relaxation, and enjoyment.",
    "Set up a bright and cheerful living room for a day spent at home with family.",
    "Create a cool and comfortable bedroom for a restful and peaceful night's sleep.",
    "Make the kitchen ready for a healthy and productive meal prep session.",
    "Prepare the outdoor area for a relaxing and enjoyable evening under the stars.",
    "Set up a cozy and romantic atmosphere for a special dinner at home with your partner.",
    "Create a festive and joyful environment for celebrating a holiday with family and friends.",
]

In [26]:
len(ambiguous_commands)

479

In [27]:
len(set(ambiguous_commands))

464

In [28]:
ambiguous_commands = list(set(ambiguous_commands))
len(ambiguous_commands)

464

### Create implicit dataset

In [30]:
def prepare_dataset(utterances, st_devices):
    dataset = []
    total_num_devices = len(st_devices)
    
    for _ in tqdm(range(0, 20)):
        for utt in tqdm(utterances):
            num_devices_context = random.randint(2, 12)
            context_devices = random.sample(sorted(st_devices), num_devices_context)

            dataset.append({"utterance": utt, "devices": context_devices})
    return dataset

In [32]:
ambiguous_dataset = prepare_dataset(ambiguous_commands + utterances, st_devices)

100%|██████████| 483/483 [00:00<00:00, 85924.79it/s]

100%|██████████| 483/483 [00:00<00:00, 84470.20it/s]

100%|██████████| 483/483 [00:00<00:00, 91825.26it/s]

100%|██████████| 483/483 [00:00<00:00, 83712.76it/s]

100%|██████████| 483/483 [00:00<00:00, 83509.16it/s]

100%|██████████| 483/483 [00:00<00:00, 83474.76it/s]

100%|██████████| 483/483 [00:00<00:00, 84119.45it/s]

100%|██████████| 483/483 [00:00<00:00, 72045.55it/s]

100%|██████████| 483/483 [00:00<00:00, 84862.97it/s]

100%|██████████| 483/483 [00:00<00:00, 81209.37it/s]

100%|██████████| 483/483 [00:00<00:00, 84948.37it/s]

100%|██████████| 483/483 [00:00<00:00, 69671.87it/s]

100%|██████████| 483/483 [00:00<00:00, 81624.92it/s]

100%|██████████| 483/483 [00:00<00:00, 83595.31it/s]

100%|██████████| 483/483 [00:00<00:00, 76111.09it/s]

100%|██████████| 483/483 [00:00<00:00, 84586.59it/s]

100%|██████████| 483/483 [00:00<00:00, 78897.41it/s]

100%|██████████| 483/483 [00:00<00:00, 58219.07it/s]

100%|██████████| 20/20 [00:0

In [33]:
len(ambiguous_dataset)

9660

In [39]:
with open('ambiguous_utterances_device_combination_data.json', 'w') as f:
    json.dump(ambiguous_dataset, f)

#### Generate text response of implicit dataset with gpt

In [62]:
def get_model_prediction(data):
    utterance = data["utterance"]
    devices = data["devices"]

    user_prompt_template = """You are a home assistant and you can perform tasks based on a user command. Here are a list of devices and \
their capabilities and a user command.

Devices: {devices_and_capabilities}

User Command: {utterance}

You need to tell how you can complete the user task with the given devices. Do not ask questions. Answer in plain text the devices you \
will use and what you will do with them. Some devices will be relevant for the command and some devices will not be relevant. You need to \
generate a response with all device. If you are using a device, mention why and how you are using it and if you are not using a device, \
mention in details why you are not using it."""

    user_prompt = user_prompt_template.format(devices_and_capabilities={d: st_devices[d] for d in devices}, utterance=utterance)
    gpt_response = openai_llm.get_completion(user_prompt, max_tokens=1000)

    return user_prompt, gpt_response

In [83]:
dataset_with_response = []
for data in tqdm(ambiguous_dataset[9000:]):
    prompt, gpt_response = get_model_prediction(data)
    
    dataset_with_response.append({
        "utterance": data["utterance"],
        "devices": data["devices"],
        "prompt": prompt,
        "response": gpt_response
    })

100%|██████████| 660/660 [2:00:47<00:00, 10.98s/it]  


In [84]:
with open('ambiguous_commands_dataset-9000-9659.json', 'w') as f:
    json.dump(dataset_with_response, f)

In [58]:
print(prompt)

You are a home assistant and you can perform tasks based on a user command. Here are a list of devices and their capabilities and a user command.

Devices: {'robotcleaner': ['cleaning_getProgress', 'cleaning_getStatus', 'cleaningMap_create', 'cleaningMap_remove', 'cleaningOption_turnOff', 'cleaningOption_turnOn', 'cleaning_pause', 'cleaning_setMode', 'cleaning_start', 'cleaning_startExclude', 'cleaning_stop'], 'dehumidifier': ['device_turnOff', 'device_turnOn', 'deviceTemperatureSetting_setValue', 'deviceTemperatureSetting_getValue', 'deviceTemperatureSetting_increase', 'deviceTemperatureSetting_decrease'], 'dishwasher': ['dishWashing_cancel', 'dishWashing_getProgress', 'dishWashing_getRemainingTime', 'dishWashing_getStatus', 'dishWashing_getSupportedCourses', 'dishWashing_pause', 'dishWashing_setCourse', 'dishWashing_start', 'dishWashing_startCourse'], 'tv': ['channel_getValue', 'channel_setByName', 'application_exit', 'application_launch', 'application_searchIn', 'brightness_decrease

In [59]:
print(gpt_response)

To prepare the home office for a day of focused work, I will utilize the following devices based on their capabilities and the requirements of creating a conducive work environment:

1. **Robotcleaner**: I will use the `cleaning_start` command to ensure the office is clean and dust-free, creating a neat workspace which is essential for focus.

2. **Dehumidifier**: Depending on the current humidity level, I might use `device_turnOn` to maintain a comfortable humidity level in the office. High humidity can be uncomfortable and distracting.

3. **Dishwasher**: This device will not be directly relevant to preparing the home office. However, starting the dishwasher with `dishWashing_start` before beginning work can ensure that clean dishes are available for breaks without interrupting focus later.

4. **TV**: I will use `device_turnOff` to ensure that the TV is off, eliminating potential distractions. If background noise is preferred for focus, I could alternatively use `application_launch`

### Generated gpt labeled data

In [86]:
with open("ambiguous_utterances_dataset_for_manual_labelling.json", "r") as f:
    ambiguous_utterances_dataset_for_manual_labelling = json.load(f)

In [87]:
ambiguous_utterances_dataset_for_manual_labelling

[{'utterance': 'Prepare the house for an energy-efficient night.',
  'devices': ['steamcloset', 'projector']},
 {'utterance': 'Create a festive and lively atmosphere for a birthday celebration.',
  'devices': ['camera', 'boiler']},
 {'utterance': 'Create a cool and comfortable bedroom for a summer night.',
  'devices': ['waterpurifier', 'blind', 'airdresser', 'ricecooker']},
 {'utterance': 'Make the dining room elegant for a formal dinner.',
  'devices': ['cooktop',
   'microwave',
   'steamcloset',
   'fan',
   'dishwasher',
   'humidifier',
   'light',
   'speaker',
   'shoedresser',
   'thermostat',
   'projector',
   'robotcleaner']},
 {'utterance': 'Tidy up the house.',
  'devices': ['waterpurifier',
   'steamcloset',
   'airconditioner',
   'robotcleaner',
   'light',
   'airpurifier',
   'fan',
   'projector',
   'oven',
   'blind']},
 {'utterance': 'Set up a comfortable and cool bedroom for a summer night.',
  'devices': ['speaker',
   'steamcloset',
   'airpurifier',
   'oven'

In [126]:
with open("Device_To_Actions.json", "r") as f:
    device_to_actions = json.load(f)

In [127]:
device_to_actions

{'airconditioner': ['device_turnOff',
  'device_turnOn',
  'airConditioningMode_getValue',
  'airConditioningMode_setValue',
  'deviceTemperatureSetting_setValue',
  'deviceTemperatureSetting_getValue',
  'deviceTemperatureSetting_increase',
  'deviceTemperatureSetting_decrease',
  'currentTemperature_getValue'],
 'airdresser': ['dressing_cancel',
  'dressing_getProgress',
  'dressing_getRemainingTime',
  'dressing_getStatus',
  'dressing_getSupportedCourses',
  'dressingMode_setValue',
  'dressing_pause',
  'dressing_start',
  'dressing_startCourse'],
 'blind': ['device_open',
  'device_openPartial',
  'device_close',
  'device_closePartial',
  'deviceState_getCloseStatus',
  'deviceState_getOpenStatus'],
 'shoedresser': ['dressing_cancel',
  'dressing_getProgress',
  'dressing_getRemainingTime',
  'dressing_getStatus',
  'dressing_getSupportedCourses',
  'dressingMode_setValue',
  'dressing_pause',
  'dressing_start',
  'dressing_startCourse'],
 'airpurifier': ['device_turnOff',
  'd

In [128]:
import uuid

In [136]:
def get_model_output_labeling(data):
    utterance = data["utterance"]
    devices = data["devices"]

    user_prompt_template = """You are a home assistant and you can perform tasks based on a user command. Your name is Bixby. \
Here are a list of devices and their capabilities and a user command.

Devices:
{devices}

Device Capabilities:
{device_capabilities}

User Command:
{utterance}

You have to create a JSON response to fulfill the user command by using the devices and their capabilities. Use only relevant \
device capabilities. Do not try to use a capability that can alter an existing operation unless the alteration is required. \
Like you can change ac to windfree mode to reduce noise from ac instead of turning it off. Each capability should have a device \
parameter with value deviceID of the device which will be used.
The JSON response should look like this:
{{"tool_calls": [{{"type": "function", "function": [{{"name": "<capability_name>", "arguments": {{"device": "<device_id>", \
<other paramerters>}}}}, ...]}}]}}

Here are a few Examples you can use to refer.
Example 1
User Command: I am getting a call, Please silence devices. Generate the plan.
Your response: {{"tool_calls": [{{"type": "function", "function": [{{"name": "mute_turnOn", "arguments": {{"device": "tv1"}}}}, \
{{"name": "airConditioningMode_setValue", "arguments": {{"device": "ac1", "mode": "WindFree"}}}}]}}]}}
Example 2
User Command: Start cleaning the living room
Your response: {{"tool_calls": [{{"type": "function", "function": [{{"name": "device_turnOff", "arguments": {{"device": \
"<fan_device_id>", "room": "living room"}}}}, {{"name": "cleaning_start", "arguments": {{"device": \
"<vaccum_cleaner_device_id>"}}}}]}}]}}

Now generate your response for the given user command using the given device capabilities. Your response
"""

    home_devices = {device: {"device_id": str(uuid.uuid4())[:5]} for device in devices}
    device_capabilities = {}
    for device in devices:
        device_capabilities[device] = {"capabilities": []}
        for action in device_to_actions[device]:
            with open(f"Griham_Actions/{action}.json", "r") as f:
                device_capabilities[device]["capabilities"].append(json.load(f))
    
    assert len(device_capabilities) == len(devices)
    user_prompt = user_prompt_template.format(devices=home_devices, device_capabilities=device_capabilities, utterance=utterance)
    gpt_response = openai_llm.get_completion(user_prompt, max_tokens=1000)

    return user_prompt, gpt_response

In [137]:
gpt_labeled_ambiguous_utterances_dataset_for_manual_labelling = []
for data in tqdm(ambiguous_utterances_dataset_for_manual_labelling[:10]):
    prompt, gpt_response = get_model_output_labeling(data)
    
    gpt_labeled_ambiguous_utterances_dataset_for_manual_labelling.append({
        "utterance": data["utterance"],
        "devices": data["devices"],
        "prompt": prompt,
        "response": gpt_response
    })

 20%|██        | 2/10 [00:06<00:25,  3.16s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'Griham_Actions/deviceState_getCloseStatus.json'

In [138]:
print(gpt_labeled_ambiguous_utterances_dataset_for_manual_labelling[0]["response"])

```json
{"tool_calls": [{"type": "function", "function": [{"name": "brightness_decreasePartial", "arguments": {"device": "63aa6"}}, {"name": "volume_setMin", "arguments": {"device": "63aa6"}}]}]}
```


In [125]:
print(gpt_labeled_ambiguous_utterances_dataset_for_manual_labelling[0]["prompt"])

You are a home assistant and you can perform tasks based on a user command. Your name is Bixby. Here are a list of devices and their capabilities and a user command.

Devices:
{'steamcloset': {'device_id': '971a0'}, 'projector': {'device_id': '72425'}}

Device Capabilities:
{'steamcloset': {'capabilities': [{'name': 'dressing_cancel', 'description': 'This function is used to cancel the clothing management function on a specified SmartThings device.', 'parameters': [{'name': 'location', 'type': 'string', 'description': 'The location where the clothing management device is located.', 'optional': True}, {'name': 'room', 'type': 'string', 'description': 'The room where the clothing management device is located.', 'optional': True}, {'name': 'device', 'type': 'string', 'description': 'The name of the clothing management device that needs to be cancelled.', 'optional': True}], 'depends_upon': ['identify_device'], 'output': []}, {'name': 'dressing_getProgress', 'description': "This function i

### Check missing action.json files in Griham_Actions folder

In [139]:
def get_missing_griham_action_files():
    missing_action_files = []
    for device in device_to_actions:
        for action in device_to_actions[device]:
            if not os.path.exists(f"Griham_Actions/{action}.json"):
                missing_action_files.append(action)
    
    print(missing_action_files)

In [140]:
get_missing_griham_action_files()

['deviceState_getCloseStatus', 'deviceState_getOpenStatus', 'cookingTimer_getRemainingTime', 'cookingTimer_increase', 'cookingTimer_setValue', 'cooking_cancel', 'cooking_getStatus', 'cooking_pause', 'cooking_setCourse', 'cookingTimer_decrease', 'cooking_cancel', 'cooking_getStatus', 'cooking_pause', 'cooking_setCourse', 'cookingTimer_decrease', 'cookingTimer_getRemainingTime', 'cookingTimer_increase', 'cookingTimer_setValue', 'cooking_cancel', 'cooking_getStatus', 'cooking_pause', 'cooking_setCourse', 'cookingTimer_decrease', 'cookingTimer_getRemainingTime', 'cookingTimer_increase', 'cookingTimer_setValue', 'cooking_cancel', 'cooking_getStatus', 'cooking_pause', 'cooking_setCourse', 'cookingTimer_decrease', 'cookingTimer_getRemainingTime', 'cookingTimer_increase', 'cookingTimer_setValue', 'deviceState_getCloseStatus', 'deviceState_getOpenStatus', 'application_searchIn', 'deviceState_getCloseStatus', 'deviceState_getOpenStatus', 'deviceState_getCloseStatus', 'deviceState_getOpenStatus']

### Compare reasoning of 13B, 32B, 7B models

In [9]:
dataset = prepare_investigation_data(utterances, st_devices)
len(dataset)

76

In [26]:
dataset_with_response = get_model_prediction(dataset, model, tokenizer)

100%|██████████| 76/76 [18:17<00:00, 14.44s/it]


In [51]:
# openai_llm.get_completion("hi")

In [59]:
def score_with_gpt(dataset_with_response):
    dataset_with_verdict = []
    for data in tqdm(dataset_with_response):
        utterance = data["utterance"]
        devices = data["devices"]
        model_response = data["response"]
        
        prompt_template = """You are a linguist. You are given a list of devices, a uer command, and an AI generated response. The AI \
assistant was asked to generate a plan to fulfill the user command. You need to categorize the AI generated response into three categories:

1) USEFUL: Use this category if the response generated by the AI assistant is usable without any modification.
2) SOMEWHAT_USEFUL: Use this category if the AI response is partially usable, i.e., it generated actions for some device that were not \
required, but it also generated actions from some device that were required.
3) NOT_USEFUL_BUT_TRAINABLE: Use this category if the AI generated a wrong response but displayed some understanding of how to solve \
the problem.
3) NOT_USEFUL:  Use this category if the AI response generated irrelevant reponse or generated a response with in any way doesn't have any \
correlation with the user command or the generated actions are completely wrong.

User Command:
{utterance}

Available Devices:
{devices}

AI Response:
{model_response}

Give your verdict in the form of a JSON. {{"verdict": <category>, "reason": <why did you select this category>}}
"""
        
        prompt = prompt_template.format(utterance=utterance, devices=devices, model_response=model_response)
        gpt_response = json.loads(openai_llm.get_completion(prompt=prompt))
        
        gpt_verdict = gpt_response["verdict"]
        gpt_verdict_reason = gpt_response["reason"]
        
        dataset_with_verdict.append({
            "utterance": utterance,
            "devices": devices,
            "prompt": data["prompt"],
            "response": model_response,
            "gpt_verdict": gpt_verdict,
            "gpt_reason": gpt_verdict_reason
        })
    return dataset_with_verdict

In [55]:
dataset_with_verdict = score_with_gpt(dataset_with_response)

In [56]:
with open('qwen-instruct-32b-response-with-verdict.json', 'w') as f:
    json.dump(dataset_with_verdict, f)

In [57]:
def print_model_response_score(dataset_with_verdict):
    score = {}
    for data in dataset_with_verdict:
        verdict = data["gpt_verdict"]
        
        if verdict not in score:
            score[verdict] = 0
        
        score[verdict] += 1
    
    print(score)

In [58]:
print_model_response_score(dataset_with_verdict)

{'USEFUL': 55, 'SOMEWHAT_USEFUL': 16, 'NOT_USEFUL_BUT_TRAINABLE': 3, 'NOT_USEFUL': 2}


### Phi-3-medium-4k 13B Instruct

In [63]:
import gc
gc.collect()
torch.cuda.empty_cache()
del model
del tokenizer

NameError: name 'model' is not defined

In [65]:
double_quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-medium-4k-instruct", quantization_config=double_quant_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-medium-4k-instruct")

config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.15k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

In [66]:
dataset_with_response = get_model_prediction(dataset, model, tokenizer)

100%|██████████| 76/76 [16:35<00:00, 13.10s/it]


In [67]:
with open('phi-3-4k-instruct-13b-response.json', 'w') as f:
    json.dump(dataset_with_response, f)

In [68]:
dataset_with_verdict = score_with_gpt(dataset_with_response)

100%|██████████| 76/76 [08:10<00:00,  6.45s/it]


In [69]:
with open('phi-3-4k-instruct-13b-response-with-verdict.json', 'w') as f:
    json.dump(dataset_with_verdict, f)

In [70]:
print_model_response_score(dataset_with_verdict)

{'SOMEWHAT_USEFUL': 37, 'USEFUL': 34, 'NOT_USEFUL_BUT_TRAINABLE': 3, 'NOT_USEFUL': 2}


### Mistral 7B Instruct v0.3

In [71]:
import gc
gc.collect()
torch.cuda.empty_cache()
del model
del tokenizer

In [72]:
double_quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained("/data/storage/models/Mistral-7B-Instruct-v0.3/", quantization_config=double_quant_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("/data/storage/models/Mistral-7B-Instruct-v0.3/")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [73]:
dataset_with_response = get_model_prediction(dataset, model, tokenizer)

100%|██████████| 76/76 [09:53<00:00,  7.81s/it]


In [74]:
with open('mistral-instruct-7b-response.json', 'w') as f:
    json.dump(dataset_with_response, f)

In [75]:
dataset_with_verdict = score_with_gpt(dataset_with_response)

100%|██████████| 76/76 [06:59<00:00,  5.51s/it]


In [76]:
with open('mistral-instruct-7b-response-with-verdict.json', 'w') as f:
    json.dump(dataset_with_verdict, f)

In [77]:
print_model_response_score(dataset_with_verdict)

{'SOMEWHAT_USEFUL': 39, 'USEFUL': 31, 'NOT_USEFUL': 4, 'NOT_USEFUL_BUT_TRAINABLE': 2}


### Mistral 7B Instruct v0.3

In [83]:
import gc
gc.collect()
torch.cuda.empty_cache()
del model
del tokenizer

In [84]:
double_quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_use_double_quant=True,
)

# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct", quantization_config=double_quant_config, device_map="auto")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
dataset_with_response = get_model_prediction(dataset, model, tokenizer)

 21%|██        | 16/76 [23:40<1:08:16, 68.27s/it] 

In [74]:
with open('mistral-instruct-7b-response.json', 'w') as f:
    json.dump(dataset_with_response, f)

In [75]:
dataset_with_verdict = score_with_gpt(dataset_with_response)

100%|██████████| 76/76 [06:59<00:00,  5.51s/it]


In [76]:
with open('mistral-instruct-7b-response-with-verdict.json', 'w') as f:
    json.dump(dataset_with_verdict, f)

In [77]:
print_model_response_score(dataset_with_verdict)

{'SOMEWHAT_USEFUL': 39, 'USEFUL': 31, 'NOT_USEFUL': 4, 'NOT_USEFUL_BUT_TRAINABLE': 2}
